# E09 Movielens 영화 추천

## 1. 데이터 준비와 전처리

In [6]:
import pandas as pd
PATH = '/content/drive/MyDrive/Github/AIFFEL/E09'

In [49]:
rating_file_path= PATH + '/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [50]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [51]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [52]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [53]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path= PATH + '/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [54]:
#merge
data = pd.merge(movies, ratings, left_on='movie_id', right_on='movie_id', how='inner')
data.head()

,movie_id,title,genre,user_id,count,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


## 2. 분석하기
ratings에 있는 유니크한 영화 개수


In [55]:
data['movie_id'].nunique()

3628

rating에 있는 유니크한 사용자 수

In [56]:
data['user_id'].nunique()

6039

가장 인기 있는 영화 30개(인기순)

In [59]:
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

유저별 평가한 영화 수

In [ ]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

## 3. 내가 선호하는 영화를 5가지 골라서 rating에 추가

In [58]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['Toy Story (1995)' , 'Jumanji (1995)' ,'Star Wars: Episode IV - A New Hope (1977)' ,'Matrix, The (1999)']

# 'jungin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['jungin']*5, 'title': my_favorite, 'rating': 5})

if not data.isin({'user_id':['jungin']})['user_id'].any():  # user_id에 'jungin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,genre,user_id,count,timestamp,rating
836473,3952.0,"Contender, The (2000)",Drama|Thriller,5682,3.0,1.029458e+09,NaN
836474,3952.0,"Contender, The (2000)",Drama|Thriller,5812,4.0,9.920721e+08,NaN
836475,3952.0,"Contender, The (2000)",Drama|Thriller,5831,3.0,9.862231e+08,NaN
836476,3952.0,"Contender, The (2000)",Drama|Thriller,5837,4.0,1.011903e+09,NaN
836477,3952.0,"Contender, The (2000)",Drama|Thriller,5998,4.0,1.001781e+09,NaN
0,NaN,Toy Story (1995),NaN,jungin,NaN,NaN,5.0
1,NaN,Jumanji (1995),NaN,jungin,NaN,NaN,5.0
2,NaN,Grumpier Old Men (1995),NaN,jungin,NaN,NaN,5.0
3,NaN,"Nightmare Before Christmas, The (1993)",NaN,jungin,NaN,NaN,5.0
4,NaN,Tom and Huck (1995),NaN,jungin,NaN,NaN,5.0


## 4. CSR matrix 만들기

In [ ]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

## 5. als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시키기

## 6.내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.


## 7.내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.



## 8. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [ ]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

In [ ]:
jungin, black_eyed_peas = user_to_idx['jungin'], artist_to_idx['black eyed peas']
jungin_vector, black_eyed_peas_vector = als_model.user_factors[jungin], als_model.item_factors[black_eyed_peas]

print('슝=3')

In [ ]:
zimin_vector

In [ ]:
black_eyed_peas_vector

In [ ]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

In [ ]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)